

# Jubaclassifierハンズオン

主に書籍に書いてある内容について、説明します。




## Agenda
- 分類とは
- 実際に動かしてみる
- 環境

### 分類とは
分類(Classifier)とは、与えられたデータに対して適切なクラスを推定する処理を指します。

- スパムメール判定
- 金融における顧客のデフォルト(不払い)予測
- etc ...


画像を載せる1

画像を載せる2

## 実際に動かしてみる

### 環境
VMで配った環境に、以下のものが入っているか確認ください。
- Jubatus(>=hoge)
- Python(3.x)
    - Jubatus
    - sklearn
    - numpy
    - pandas

### データ
**default of credit card clients Data Set**

- 台湾の顧客に対し支払いの不払い(デフォルト)があったかどうかを集めたデータ
- 年齢や性別など、23個の特徴量で構成されている

※データの構造や型に合わせて処理を施しているので、本の内容と少し異なる箇所もありますが、流れは本のままで進めます。

- `DEFAULT`に支払いが履行されたかどうかの情報が含まれている
- `DEFAULT`を分類器の答え合わせ（正解ラベル)に用いる

|列番|特徴量|概要|特徴量の型|
|:--|:--|:--|:-|
|X1|Amount of the given credit|信用貸付額|整数|
|X2|Gender|性別|カテゴリ変数|
|X3|Education|学歴|カテゴリ変数|
|X4|Martial status|結婚歴|カテゴリ変数|
|X5|Age|年齢|整数|
|X6-X11|History of past payment|過去の支払い。きちんと支払ったかどうか(※1)|整数|
|X12-X17|Amount of bill statememt|過去の請求額(※1)|整数|
|X18-X23|Amount of previous payment|過去の支払額(※1)|整数|
|Y|DEFAULT|デフォルトしたかどうか|カテゴリ変数（正解ラベル）|


※1 2005年4月から2005年9月までの６ヶ月分のデータ

In [36]:
# データを読み込んでみる
import pandas as pd
df = pd.read_csv("data/default_train.csv")
print(df.head())


       X1  X2  X3  X4  X5  X6  X7  X8  X9  X10 ...    X15    X16    X17   X18  \
0   20000   2   2   1  24   2   2  -1  -1   -2 ...      0      0      0     0   
1  120000   2   2   2  26  -1   2   0   0    0 ...   3272   3455   3261     0   
2   90000   2   2   2  34   0   0   0   0    0 ...  14331  14948  15549  1518   
3   50000   2   2   1  37   0   0   0   0    0 ...  28314  28959  29547  2000   
4   50000   1   2   1  57  -1   0  -1   0    0 ...  20940  19146  19131  2000   

     X19    X20   X21   X22   X23  Y  
0    689      0     0     0     0  1  
1   1000   1000  1000     0  2000  1  
2   1500   1000  1000  1000  5000  0  
3   2019   1200  1100  1069  1000  0  
4  36681  10000  9000   689   679  0  

[5 rows x 24 columns]


## Jubatusを起動
- ターミナルに戻り、下記コマンドを入力します
- まずは線形分類器(AROW)を使ってみます

```
$ jubaclassifier -f config/linear.json&
```

### Column1 Jubatus分類器に入っているアルゴリズム

Jubatusでは、線形分類器と非線形分類器にそれぞれ下表のアルゴリズムを用意しています。

[表]

### Column2 Reguralization Parameterとは

## データの読み込み
- pandasを用いてデータを読み込む
- 特徴量ベクトル, ラベル, 特徴量の名前をcsvから出力

In [37]:
def read_dataset(path):
    df = pd.read_csv(path)
    labels = df['Y'].tolist()
    df = df.drop('Y', axis=1)
    features = df.as_matrix().astype(float)
    columns = df.columns.tolist()
    return features, labels, columns

features_train, labels_train, columns = read_dataset('data/default_train.csv')

In [38]:
print("columns : {}".format(columns))
print("features : {}".format(features_train))

columns : ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23']
features : [[  2.00000000e+04   2.00000000e+00   2.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.20000000e+05   2.00000000e+00   2.00000000e+00 ...,   1.00000000e+03
    0.00000000e+00   2.00000000e+03]
 [  9.00000000e+04   2.00000000e+00   2.00000000e+00 ...,   1.00000000e+03
    1.00000000e+03   5.00000000e+03]
 ..., 
 [  1.60000000e+05   1.00000000e+00   6.00000000e+00 ...,   4.50000000e+03
    4.50000000e+03   4.50000000e+03]
 [  8.00000000e+04   1.00000000e+00   2.00000000e+00 ...,   5.52000000e+03
    0.00000000e+00   2.99800000e+03]
 [  2.10000000e+05   1.00000000e+00   3.00000000e+00 ...,   1.00000000e+03
    1.80000000e+03   1.40150000e+04]]


## 学習用データをDatum形式に変換
- Jubatusにデータを投げるために、pandasで読み込んだデータをDatum形式にする必要がある
- データと正解ラベルを1セットにして、次のような形でデータを保持している

```
[(正解ラベル, Datum(key, value))]
```

In [39]:
features_train, labels_train, columns = read_dataset('data/default_train.csv')
train_data = []
for x, y in zip(features_train, labels_train):
    d = Datum({key: float(value) for key, value in zip(columns, x)})
    train_data.append([str(y), d])

## 作成したデータをJubatusサーバに投入

In [41]:
from jubatus.classifier.client import Classifier
client = Classifier('127.0.0.1', 9199, '') # Jubatusサーバのホストとポートを指定する
client.clear() #過去の学習結果を一度初期化する(任意)
client.train(train_data) # 学習を実行

24998

## モデルを用いて分類/評価をする
- 作った学習モデルを使って、実際に分類をしてみる
- `data/default_test.csv`を用いる

※実際の評価の際は、CrossValidationなどを用いて評価を行うのが良いでしょう。

In [42]:
# テスト用Datumリストを作る
features_test, labels_test, columns = read_dataset('data/default_test.csv')
test_data = []
for x, y in zip(features_test, labels_test):
    d = Datum({key: float(value) for key, value in zip(columns, x)})
    test_data.append(d)

## テストをする
実際にJubaclassifierにデータを投入し、返ってくるラベルを見る

In [43]:
# テストをする
results = client.classify(test_data)

Jubatusがdefaultに対してyes/no どちらが可能性が高いかをスコアリングしてくれている

In [57]:
for result in results[:10]:
    print(result)

[estimate_result{label: 1, score: 0.3523704707622528}, estimate_result{label: 0, score: 0.909548819065094}]
[estimate_result{label: 1, score: 6.473033428192139}, estimate_result{label: 0, score: 6.9137773513793945}]
[estimate_result{label: 1, score: 1.4632307291030884}, estimate_result{label: 0, score: 10.985992431640625}]
[estimate_result{label: 1, score: -0.8135554790496826}, estimate_result{label: 0, score: 2.7254507541656494}]
[estimate_result{label: 1, score: 5.496100902557373}, estimate_result{label: 0, score: 6.213317394256592}]
[estimate_result{label: 1, score: 0.7540934681892395}, estimate_result{label: 0, score: 1.3192205429077148}]
[estimate_result{label: 1, score: 3.4982974529266357}, estimate_result{label: 0, score: 3.661841630935669}]
[estimate_result{label: 1, score: 0.5923149585723877}, estimate_result{label: 0, score: 0.8364338874816895}]
[estimate_result{label: 1, score: 3.833550453186035}, estimate_result{label: 0, score: 3.8580896854400635}]
[estimate_result{label: 

## 結果の分析
- 先ほどのスコアの大きい方を分類結果として返す`get_most_likely`関数を作る

In [58]:
# 結果を分析する
def get_most_likely(result):
    return max(result, key = lambda x: x.score).label

### 分類結果の評価指標
- 分類結果は、**正例か負例か**という観点と、**答えが合ってたかどうか**という観点で４種類に分けられる
- 実際のデータが。。。
    - Defaultしている : 正例(Positive)
    - Defaultしていない : 負例(Negative)
- 推定結果が。。。
    - 正しい : True
    - 間違ってる : False
    

### 混合行列(Confusion Matrix)
この4種類を一つの表にまとめたもの

[図]


In [66]:
def analyze_results(labels, results, pos_label="1", neg_label="0"):
    tp, fp, tn, fn = 0, 0, 0, 0
    for label, result in zip(labels, results):
        estimated = get_most_likely(result)
        label = str(label)
        estimated = str(estimated)
        if label == pos_label and label == estimated:
            tp += 1
        elif label == pos_label and label != estimated:
            fn += 1
        elif labels != pos_label and label == estimated:
            tn += 1
        else:
            fp += 1
    accuracy = float(tp + tn) / float(tp + tn + fp + fn)
    precision = float(tp) / float(tp + fp) 
    recall = float(tp) / float(tp + fn)
    f_value = 2.0 * recall * precision / (recall + precision)
    # confusion matrix
    confusion = pd.DataFrame([[tp, fp], [fn, tn]], index=[pos_label, neg_label], columns=[pos_label, neg_label])
    return confusion, accuracy, precision, recall, f_value

In [68]:
confusion, accuracy, precision, recall, f_value = analyze_results(labels_test, results)
print('confusion matrix\n{0}\n'.format(confusion))
print('metric    : score')
print('accuracy  : {0:.3f}'.format(accuracy))
print('precision : {0:.3f}'.format(precision))
print('recall    : {0:.3f}'.format(recall))
print('f_value   : {0:.3f}'.format(f_value))

confusion matrix
     1     0
1  308   234
0  751  3708

metric    : score
accuracy  : 0.803
precision : 0.568
recall    : 0.291
f_value   : 0.385


### Recallが低い問題
- Accuracyが高いにも関わらず、**Recallが低い**
- Recall : **Defaultする人に対し、Defaultしたと予測できているか**
    - これが低い => Defaultするはずの人を捉えられていない => リスク管理できない！

|metric|score|
|---|---|
|Accuracy|**0.803**|
|Precision|0.568|
|Recall|**0.291**|
|f_value|0.385|



### Recallを上げるために => アンダーサンプリング
- 現状
    - 正例(defaultした) : hogehoge件
    - 負例(defaultしていない) : hogehgeo件
- **負例に引きずられて、分類がうまくいかない可能性あり**


In [79]:
# シードで乱数を固定(再現性を得たい場合に実行)
import random
random.seed(42)
def under_sampling(features, labels, reduce_label, reduce_rate=0.125):
    sampled_features, sampled_labels = [], []
    for feature, label in zip(features, labels):
        label = str(label)
        if label != reduce_label or random.random() < reduce_rate:
            sampled_features.append(feature)
            sampled_labels.append(label)
    return sampled_features, sampled_labels
# アンダーサンプリング
sampled_features_train, sampled_labels_train = under_sampling(features_train, labels_train, reduce_label='0')

In [80]:
# アンダーサンプリング
sampled_features_train, sampled_labels_train = under_sampling(features_train, labels_train, reduce_label='0')
# 学習用Datumリストを作る
sampled_train_data = []
for x, y in zip(sampled_features_train, sampled_labels_train):
    d = Datum({key: float(value) for key, value in zip(columns, x)})
    sampled_train_data.append([str(y), d])
# 学習をする
client = Classifier('127.0.0.1', 9199, '')
client.clear()
client.train(sampled_train_data)
# テストをする
results = client.classify(test_data)


confusion matrix
     1     0
1  912  2760
0  147  1182

metric    : score
accuracy  : 0.419
precision : 0.248
recall    : 0.861
f_value   : 0.386


In [81]:
# 結果を分析する
confusion, accuracy, precision, recall, f_value = analyze_results(labels_test, results)
print('confusion matrix\n{0}\n'.format(confusion))
print('metric    : score')
print('accuracy  : {0:.3f}'.format(accuracy))
print('precision : {0:.3f}'.format(precision))
print('recall    : {0:.3f}'.format(recall))
print('f_value   : {0:.3f}'.format(f_value))

confusion matrix
     1     0
1  912  2760
0  147  1182

metric    : score
accuracy  : 0.419
precision : 0.248
recall    : 0.861
f_value   : 0.386
